In [ ]:
!pip install -qqq autogen groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.1/350.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 933.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

GROQ_API_KEY=userdata.get('GROQ_API_KEY')

In [ ]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

temp_dir=tempfile.TemporaryDirectory()
config_list = [
    {
        # Let's choose the Llama 3 model
        "model": "llama-3.1-70b-versatile",
        # Put your Groq API key here or put it into the GROQ_API_KEY environment variable.
        "api_key": GROQ_API_KEY,
        # We specify the API Type as 'groq' so it uses the Groq client class
        "api_type": "groq",
    }
]

executer=LocalCommandLineCodeExecutor(timeout=20,work_dir=temp_dir.name)

code_writer_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.

- you are given a str path. you should read the JSON file and optimize the inventory cost.
- There are 34 ingredents in the inventory.
- datset is like:
{
    "products": [
      {
        "name": "Espresso",
        "sell_price":1.40,
        "sale_ratio":"10%",
        "ingredients": [
          {"name": "Espresso beans", "quantity": 22, "unit":"grams","buying_price":17.3, "buying_unit": "1000g/pack"},
          {"name": "Water", "quantity": 60,"unit":"ml","buying_price":0.5,"buying_unit": "1500ml/bottle"}
        ]
      },
      {
        "name": "Americano",
        "sell_price":1.50,
        "sale_ratio":"4%",
        "ingredients": [
          {"name": "Espresso beans", "quantity": 22, "unit":"grams","buying_price":17.3,"buying_unit": "1000g/pack"},
          {"name": "Water", "quantity": 180,"unit":"ml","buying_price":0.5,"buying_unit": "1500ml/bottle"}
        ]
      },
      }
- Your task is to calculate the minimum number of packs required for each of 34 ingredients, ensuring all fit within the specified budget.
- Each ingredient has a fixed buying price per pack (e.g., $1.3 for 1000ml of milk or $11.2 for a 500ml pack of caramel sauce).
- For each ingredient, calculate the number of packs needed to stay within the budget constraints, while ensuring every ingredient has at least one pack (minimum pack count is 1).
- Do not make up anything by youeself. Every ingredients should exactly match.
### Output Format:
- Please provide the output as a JSON file of ingredients and their pack counts in this exact format:
  `Ingredient1: PackCount1, Ingredient2: PackCount2, ..., Ingredient34: PackCount34`
- The output must include exactly 34 ingredients with their pack counts (no omissions).
- create a JSON file

### Budget Constraints:
- The budget must be respected exactly, using the equation:
  `buying_price1 * pack_count1 + buying_price2 * pack_count2 + ... + buying_price34 * pack_count34 <= Budget`

### Additional Guidelines:
- Distribute the budget efficiently to minimize excess pack counts.
- Ensure all pack counts are integers, with no zero values.

"""

code_writer_agent=ConversableAgent(
    'code_writer',
    llm_config={"config_list": config_list},
    system_message=code_writer_system_message,
    code_execution_config=False

)

code_executer_agent=ConversableAgent(
    'code_executer',
    llm_config=False,
    code_execution_config={'executor':executer},
    human_input_mode='ALWAYS',
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"]
)


In [ ]:
chat_result = code_executer_agent.initiate_chat(
    code_writer_agent,
    message=f"Here is the path: /content/Full_Dataset.json, after that you done your job  save the JSON to a file named 'inventory.json",

)

code_executer (to code_writer):

Here is the path: /content/Full_Dataset.json, after that you done your job  save the JSON to a file named 'inventory.json

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executer):

First, I need to read the JSON file and parse the data. Then, I will calculate the minimum number of packs required for each ingredient while staying within the budget constraints. 

Please execute the following code to perform these tasks:

```python
# filename: calculate_inventory.py
import json

def calculate_packs(budget, buying_price, buying_unit):
    quantity_per_pack = int(buying_unit.split('/')[0])
    unit = buying_unit.split('/')[1].replace('g', '').replace('ml', '')
    return max(1, (budget / buying_price) * (quantity_per_pack / 1000)) // 1 + 1 if buying_price / quantity_per_pack > budget / 1000 else (budget / buying_price) // 1

def main():
    # Given JSON file path
    json_f

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: change the budget=2500
code_executer (to code_writer):

change the budget=2500

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executer):

Here is the updated code:

```python
# filename: calculate_inventory.py
import json

def calculate_packs(budget, buying_price, buying_unit):
    quantity_per_pack = int(buying_unit.split('/')[0])
    unit = buying_unit.split('/')[1].replace('g', '').replace('ml', '')
    return max(1, (budget / buying_price) * (quantity_per_pack / 1000)) // 1 + 1 if buying_price / quantity_per_pack > budget / 1000 else (budget / buying_price) // 1

def main():
    # Given JSON file path
    json_file_path = '/content/Full_Dataset.json'
    # Given budget
    budget = 2500  # updated budget

    # Load JSON data
    with open(json_file_path, 'r')

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: just use buying_price, the buying_unit is not important
code_executer (to code_writer):

just use buying_price, the buying_unit is not important

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executer):

Here is the updated code:

```python
# filename: calculate_inventory.py
import json

def main():
    # Given JSON file path
    json_file_path = '/content/Full_Dataset.json'
    # Given budget
    budget = 2500  # updated budget

    # Load JSON data
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    # Create a dictionary to store ingredients and their pack counts
    inventory = {}

    # Iterate over products and ingredients
    for product in data['products']:
        for ingredient in product['ingredients']:
            name = ingredient

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.1-70b-versatile
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.


In [ ]:
import os

print(os.listdir(temp_dir.name))

['inventory.json', 'optimize_inventory.py', 'calculate_inventory.py']


In [ ]:
import json

file=os.path.join(temp_dir.name,'inventory.json')
with open(file, 'r') as f:
    data = json.load(f)

In [ ]:
print(data)

{'Espresso beans': 5, 'Water': 148, 'Espresso': 148, 'Milk': 57, 'Chocolate syrup': 6, 'Ice-cream': 16, 'Caramel sauce': 7, 'Black tea loose leaf': 12, 'Green tea loose leaf': 11, 'Dried hibiscus flowers': 4, 'Dried mint leaf': 16, 'Honey': 41, 'Dried ginger root': 62, 'All-purpose flour': 13, 'Sugar': 62, 'Cocoa powder': 6, 'Baking powder': 92, 'Baking soda': 10, 'Salt': 148, 'Eggs': 26, 'Vegetable oil': 32, 'Vanilla extract': 6, 'Butter': 50, 'Lemon juice': 39, 'Ground cinnamon': 12, 'Ground ginger': 9, 'Grated carrots': 17, 'Cream': 19, 'Strawberries': 14, 'Graham cracker crumbs': 7, 'Cream cheese': 50, 'Banana': 74, 'Yogurt': 34, 'Mango': 17, 'Peanut butter': 9}


In [ ]:
with open('inventory_optimization.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)